![Py4Eng](img/logo.png)

# __Iteration__

Iteration is a huge deal in programming and especially in Python, where it is one of the biggest differences between Python 3 and Python 2, and between Python 3 and some other languages, like MATLAB. 

This is a challenging session, but we will start it off with a relatively simple subject and slowly increase the difficulty level.

Some definitions before we begin:

- _iterable_ - an object that you can iterate over, for example with a `for` loop.
- _container_ - an object that contains other objects, you can use `in` on it.
- _element, item_ - an object that is contained by a container or produced by iterating on an iterable.
- _sequence_ - an ordered container.
- _iterator_ - an object that provides an iteration over an iterable; one iterable can have several, independent, non-identical iterators.

Note that not every container is iterable (i.e. [Bloom filter](https://en.wikipedia.org/wiki/Bloom_filter)) and not every iterable is a container (for example, an iterable that provides random numbers). So we can call the union of iterables and containers a **collection**.

# Comprehensions

Comprehensions are a compact way to process all or part of the elements in an iterable and return a new container.

## List comprehensions

Say we have a bunch of measurements in a list called `data` and we want to calculate the mean and the standard deviation.

The usual way to do this is with `for` loops:

In [ ]:
data = [1, 7, 3, 6, 9, 2, 7, 8]

mean = sum(data) / len(data)

print('Mean:', mean)

deviations = []
for x in data:
    deviations.append((x - mean)**2)
    
var = sum(deviations) / len(deviations)
stdev = var**0.5

print("Standard deviation:", stdev)

We can replace the `for` loop with a **list comprehension**:

In [ ]:
data = [1, 7, 3, 6, 9, 2, 7, 8]
mean = sum(data) / len(data)

# list comprehension:
deviations = [ (x - mean)**2 for x in data ]

var = sum(deviations) / len(deviations)
stdev = var**0.5
print("Standard deviation:", stdev)

We can add a condition, too. Say you want to calculate the deviations of above-average data:

In [ ]:
pos_devs = [(x - mean)**2 for x in data if x > mean]
pos_devs

Some more examples:

In [ ]:
a = [i**2 for i in range(10)]
print(a)
b = [i**2 for i in range(10) if i%2==0]
print(b)

Some performence comparisons (see more details [here](https://wiki.python.org/moin/PythonSpeed/PerformanceTips#Loops)):

In [ ]:
data = [1, 7, 3, 6, 9, 2, 7, 8] * 10000 # make a big list

In [ ]:
%%timeit 
deviations = []
for x in data:
    deviations.append((x - mean)**2)

In [ ]:
%%timeit
deviations = [(x - mean)**2 for x in data]

So the list comprehension is ~20% faster.

__Why??__

First, local variables are faster.

In [ ]:
def get_deviations(data, mean):
    deviations = []
    for x in data:
        deviations.append((x - mean)**2)
    return deviations

In [ ]:
%%timeit 
deviations = get_deviations(data, mean)

Second, method lookup adds an overhead:

In [ ]:
def get_deviations2(data, mean):
    deviations = []
    append = deviations.append # cache the method to a variable
    for x in data:
        append((x - mean)**2)
    return deviations

In [ ]:
%%timeit 
deviations = get_deviations2(data, mean) 

We get the same optimization as a list comprehension by using local vars and caching method lookups.

## Exercise: leap year

Remember that a leap is is a year that is divisible by 400 or divisible by 4 but not by 100.

Write a listcomp to build a list of all leap years until today. Print the length of the list.

In [ ]:
# your code here

### Cartesian product

We can insert several `for` statements in a single listcomp, producing all elements of a cartesian product.

For example, consider how we construct a full pack of cards:

In [ ]:
suit = ["♠", "♣", "♥", "♦"] # equivalent code: ["\u2660", "\u2666", "\u2665", "\u2663"]
rank = [str(i) for i in range(2, 11)]
rank += ['J', 'Q', 'K', 'A']
deck = [n+s for s in suit for n in rank]
print(deck)

## Dictionary comprehensions

We can also use comprehensions to create dictionaries:

In [ ]:
data_devs = {x: (x - mean)**2 for x in data}
data_devs

## Bonus: Set comprehensions

In [ ]:
with open('../data/Gulliver.txt') as f:
    txt = f.read()
unique_words = {w.lower() for w in txt.split() if w.isalpha()}

print(len(unique_words))
print('love' in unique_words)
print('war' in unique_words)
print('python' in unique_words)
print('selfie' in unique_words)

If the file is very big we could do it line by line and update the set (it's mutable):

In [ ]:
unique_words = set()
with open('../data/Gulliver.txt') as f:    
    for line in f:
        line_unique_words = {w.lower() for w in line.split() if w.isalpha()}
        unique_words.update(line_unique_words)

print(len(unique_words))
print('love' in unique_words)
print('war' in unique_words)
print('python' in unique_words)
print('selfie' in unique_words)

# Iterators

From [Python docs](https://docs.python.org/3.5/library/stdtypes.html#iterator-types):

> Python supports a concept of iteration over containers. This is implemented using two distinct methods; these are used to allow user-defined classes to support iteration. Sequences always support the iteration methods.

- `__next__`: An iterator method, returns the next item from the iteration when the iterator is called with `next(...)`. If there are no more items this method should raise `StopIteration`.

- `__iter__`: An iterable method, returns an iterator. If implemented in an iterator, it should return the iterator itself.

The easiest way to implement iterators is using generators rather than defining new iterator objects, so that's our next topic.

# Generator expressions

Generator expressions look similar to list comprehensions but really they return an iterator rather than a new collection.

For example, if we are not interested in the deviations but only in their sum, we don't have to build the actual `list`, like the comprehension does, but rather we can use a generator, which produces each value as we need it, using **lazy evaluation**. 

Note the use of `()` instead of `[]`:

In [ ]:
data = [1, 7, 3, 6, 9, 2, 7, 8]

deviations = ((x - mean)**2 for x in data)

var = sum(deviations) / len(data)
print("Standard deviation:", var**0.5)

In [ ]:
type(deviations)

Iterating over a generator can be done by calling `next`:

In [ ]:
deviations = ((x - mean)**2 for x in data)
next(deviations)

In [ ]:
next(deviations)

Next will fail with `StopIteration` when there are no more elements in the generator:

In [ ]:
len(data)

In [ ]:
next(deviations) # 3
next(deviations) # 4
next(deviations) # 5
next(deviations) # 6
next(deviations) # 7
next(deviations) # 8

In [ ]:
next(deviations)

When going over many elements, usually we don't need the list in the memory, just one number at a time. 

#### __This is where generators shine.__

For example, the `range` function actually returns a generator, which can be converted into a list:

In [ ]:
range(10)

In [ ]:
list(range(10))

If we were to go over all the numbers from 0 to $10^8$, creating that list before the iteration is costly in space. 

A generator doesn't create the entire list in memory, but rather lazily creates each elements as it is needed.

We load [ipython_memory_usage](https://github.com/ianozsvald/ipython_memory_usage), a package that monitors memory usage inside the notebook.

Install with `pip install ipython_memory_usage`, and on Windows (and also recommended on macOS) you will also need to `conda install psutil`.

In [ ]:
! conda install psutil

In [ ]:
import ipython_memory_usage.ipython_memory_usage as imu
imu.start_watching_memory()

In [ ]:
lst = list(range(10**8))

In [ ]:
rng = range(10**7)

In [ ]:
del lst
imu.stop_watching_memory()

Another example:

In [ ]:
%timeit -n 3 [x for x in range(1, 10**6) if x % 2 == 0]

In [ ]:
%timeit -n 3 (x for x in range(1, 10**8) if x % 2 == 0)

In [ ]:
%timeit -n 3 list(x for x in range(1, 10**6) if x % 2 == 0)

## Exercise: secret


Given in the code below is a dictionary (named `code`) where the keys represent encrypted characters and the values are the corresponding decrypted characters. 

Use the dictionary to decrypt an ecnrypted message (named `secret`) and print out the resulting cleartext message.

* Use a generator expression to decrypt the secret message using the code dictionary. 
* Make sure you aren't creating any lists in intermediate steps. 
* Experiment with newlines inside `( )` to improve readability.

In [ ]:
secret = """Mq osakk le eh ue usq qhp, mq osakk xzlsu zh Xcahgq,
mq osakk xzlsu eh usq oqao ahp egqaho,
mq osakk xzlsu mzus lcemzhl gehxzpqhgq ahp lcemzhl oucqhlus zh usq azc, mq osakk pqxqhp ebc Zokahp, msauqjqc usq geou dat rq,
mq osakk xzlsu eh usq rqagsqo,
mq osakk xzlsu eh usq kahpzhl lcebhpo,
mq osakk xzlsu zh usq xzqkpo ahp zh usq oucqquo,
mq osakk xzlsu zh usq szkko;
mq osakk hqjqc obccqhpqc, ahp qjqh zx, mszgs Z pe heu xec a dedqhu rqkzqjq, uszo Zokahp ec a kaclq iacu ex zu mqcq obrfblauqp ahp ouacjzhl, usqh ebc Qdizcq rqtehp usq oqao, acdqp ahp lbacpqp rt usq Rczuzos Xkqqu, mebkp gacct eh usq oucbllkq, bhuzk, zh Lep’o leep uzdq, usq Hqm Meckp, mzus akk zuo iemqc ahp dzlsu, ouqio xecus ue usq cqogbq ahp usq kzrqcauzeh ex usq ekp."""

code = {'w': 'x', 'L': 'G', 'c': 'r', 'x': 'f', 'G': 'C', 'E': 'O', 'h': 'n', 'O': 'S', 'y': 'q', 'R': 'B', 'd': 'm', 'f': 'j', 'i': 'p', 'o': 's', 'g': 'c', 'a': 'a', 'u': 't', 'k': 'l', 'q': 'e', 'r': 'b', 'V': 'Z', 'X': 'F', 'N': 'K', 'B': 'U', 'T': 'Y', 'M': 'W', 'U': 'T', 'm': 'w', 'C': 'R', 'J': 'V', 't': 'y', 'S': 'H', 'v': 'z', 'e': 'o', 'D': 'M', 'p': 'd', 'K': 'L', 'A': 'A', 'P': 'D', 'l': 'g', 's': 'h', 'W': 'X', 'H': 'N', 'j': 'v', 'z': 'i', 'I': 'P', 'b': 'u', 'Z': 'I', 'F': 'J', 'Y': 'Q', 'Q': 'E', 'n': 'k'}



## Can I iterate on it?

A generator expression creates a new generator object which implements the iterator protocol, and will be an instance of `collections.Iterator`.

However, the best way to check if something is iterable is to try to get an iterator from it and catch the `TypeError` if it can't be iterated:

In [ ]:
iter(range(10))

iter(x**2 for x in range(10))

iter(5)

But we can also check if something is an instance of `collections.Iterable` (this is a _peak before you leap_ approach):

In [ ]:
from collections.abc import Iterable

In [ ]:
print(isinstance(range(10), Iterable))

print(isinstance((x**2 for x in range(10)), Iterable))

print(isinstance([1,2,'a'], Iterable))

print(isinstance({1,2,'a'}, Iterable))

print(isinstance('abcdefg', Iterable))

print(isinstance(5, Iterable))

# Bonus: Generator functions

We can write more complex generators using functions in which the `yield` statement replaces the `return` statement. 

These are **generator functions**, which are special because:
- When calling these functions, we don't get a return value but rather a generator object which implements the iterator protocol
- Generator functions can be thought of as function that yield a value, suspend, and can then be resumed when further values or further action is required

Generator functions can provide very flexible iterations, including potentially infinite ones, and can be used for other stuff like creating context managers.

## Infinite iteration

Say we want to go over all the natural numbers to find a number that matches some condition.

We write a generator for the natural numbers, which is kind of a non-limit `range` (note that this can be done using `itertools.count` from the standard library):

In [ ]:
def natural_numbers():
    n = 0
    while True:
        n += 1
        yield n

gen = natural_numbers()
print(gen)

In [ ]:
from collections.abc import Iterator
hasattr(gen, '__iter__'), hasattr(gen, '__next__'), isinstance(gen, Iterator)

We can consume values from the generator one by one using the `next` function:

In [ ]:
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

 or by giving it to a `for` loop

In [ ]:
for n in natural_numbers():
    if n % 667 == 0 and n**2 % 766 == 0:
        break
print(n)

Of course, this can only be done using a generator - we cannot create a list of all natural numbers...

Let's do another small example to understand how a generator works:

In [ ]:
def example_generator():
    print("Start.")
    for x in range(3):
        print("Next.")
        yield x
    print("Done.")
    
for x in example_generator():
    print(x)

In [ ]:
exgen = example_generator()

In [ ]:
x = next(exgen)
print(x)

## Exercise: factors

Write a generator function that yields the factors of a given number `n`, starting from the smallest factor.

In [ ]:
def factors(n):
    # your code here

The following code prints the factors of a large numbers without ever actually holding the list of factors.

In [ ]:
n = 22324383
print(n, end= ' = ')
for k in factors(n):
    print(k, end=' * ')
print('\b\b') # remove the final '* '

## Exercise: Fibonacci

Create a generator that returns numbers from the Fibonacci series, defined by:

$$
a_0 = 0 \\
a_1 = 1 \\
a_n = a_{n-2} + a_{n-1}
$$

In [ ]:
def fibonacci():
    # your code here

In [ ]:
for n in fibonacci():
    print(n, end=' ')
    if n > 1000: break

## Generators as context manager

A "real" context manager does two things: 
* sets something up when you enter the context, maybe providing some return value
* tears something down when you exit the context. 

The most common case is:

```py
with open(filename) as f:
    # do something with file f
```
in which `open` creates a file-like object that implements the context manager interface: on entering the context it opens the file and returns itself (the file-like object); when exiting the context it closes the file.

A generator is great for that because use the `yield` statement to return a value and suspend the generator, thus the code until the `yield` implements "entering" the context; the code after the `yield` implements "exiting" the context, and Python can call `next` once when entering and again when exiting.

Let's do an example - we'll write a `tictoc` context manager that measures and prints how much time we were inside the context (similar to MATLAB's `tic; toc;` idiom).

In [ ]:
import time
import contextlib

In [ ]:
def tictoc():
    tic = time.time()
    yield
    toc = time.time()
    print("Elapsed time: {} seconds".format(toc - tic))

tictoc = contextlib.contextmanager(tictoc)

In [ ]:
with tictoc():
    list(factors(231232221))

Note that to tell Python that `tictoc` should be used as a context manager we need to apply the `contextmanager` decorator.

`contextmanager` wraps `tictoc` with some neccesary boilerplate to convert it from a generator function to a context manager factory.

A nicer, equivalent way to do this is:

In [ ]:
@contextlib.contextmanager
def tictoc():
    tic = time.time()
    yield
    toc = time.time()
    print("Elapsed time: {} seconds".format(toc - tic))

with tictoc():
    list(factors(231232221))

## Bonus: Pass values into generators

We said that generators are functions that can yield a value, suspend, and then be resumed by calling `next` - which can be done outside of an iteration context (`for` loop). Calling `next` on a generator object will give the next result of the generator (run generator to next `yield` statement):

In [ ]:
naturals = natural_numbers()
next(naturals), next(naturals), next(naturals), next(naturals)

If we can resume the generator, why not resume it with some value?

`send` allows us to **pass values into generators**. For example, say we want to iterate over the natural numbers but make a jump from one value to the next, and these jumps are not constant (can't be arguments of the generator function):

In [ ]:
def jumping_natural_numbers():
    n = 0
    while True:
        jump = yield n 
        if jump == None:
            print('hi')
            jump = 1
        n += jump

In [ ]:
gen = jumping_natural_numbers()
gen.send(None)

## Example: running average

A more whole example is given by the following generator that calculates a running average of a stream of numbers fed to it. 

It does so by keeping just three numbers - the sum, the count (number of numbers), and the average.

Each time a new number is sent, it yields the updated average and then waits for another number to be sent.

This is much more effcient in terms of space then actually keeping all the numbers; also, it's great for cases in which we actually have a stream of numbers.

In [ ]:
def running_average():
    summ = 0
    count = 0
    avg = 0
    
    while True:
        n = yield avg
        count += 1
        summ += n
        avg = summ / count

In [ ]:
import random

avg = running_average()
avg.send(None) # initialize the generator by sending None or by next(avg)

# mimick a stream of random integers between 0 and 9, inclusive
random_stream = (random.randint(0, 9) for _ in range(10))
for n in random_stream:
    current_avg = avg.send(n)
    print('{:d}: {:.4f}'.format(n, current_avg))

See more at Jeff Knupp's [blog](http://www.jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/).

## Exercise: have seen

Write a generator `have_seen` that accepts a string via `send` and yields a boolean in response.

The function will yield `True` if the input string was already seen by the function, and `False` otherwise. The function should be case insensitive.

Use it to find all the words that appear more than once in the following text (the first paragraph from [Gulliver's Travels](https://ia801404.us.archive.org/2/items/gulliverstravels17157gut/17157.txt)).

This exercise follows [Readable Python coroutines](http://takluyver.github.io/posts/readable-python-coroutines.html) by Thomas Kluyver. See solution in [have_seen.py](../solutions/have_seen.py).

In [ ]:
def have_seen():
    pass

In [ ]:
text = """My father had a small estate in Nottinghamshire; I was the third of five
sons. He sent me to Emmanuel College in Cambridge at fourteen years old,
where I resided three years, and applied myself close to my studies;
but the charge of maintaining me, although I had a very scanty
allowance, being too great for a narrow fortune, I was bound apprentice
to Mr. James Bates, an eminent surgeon in London, with whom I continued
four years; and my father now and then sending me small sums of money, I
laid them out in learning navigation, and other parts of the mathematics
useful to those who intend to travel, as I always believed it would be,
some time or other, my fortune to do. When I left Mr. Bates, I went down
to my father, where, by the assistance of him, and my uncle John and
some other relations, I got forty pounds, and a promise of thirty
pounds a year, to maintain me at Leyden. There I studied physic two
years and seven months, knowing it would be useful in long voyages."""
text = text.lower().split()


# Functional programming

## Map

A map applies a function on all elements of an iterable.

In [ ]:
poets = [
    'Shel Silverstein', 
    'Pablo Neruda', 
    'Maya Angelou',
    'Edgar Allan Poe',
    'Robert Frost',
    'Emily Dickinson',
    'Walt Whitman'
]

first_names = (name.partition(' ')[0] for name in poets)
first_names

In [ ]:
for n in first_names:
    print(n)

There's a dedicated `map` function in Python that works very similary, but requires an explicit function defined.

In [ ]:
def get_first_name(name):
    return name.partition(' ')[0]
first_names = map(get_first_name, poets)
first_names

In [ ]:
list(first_names)

## Filter

A filter iterates over the elements in an iterable that pass a certain test or predicate.

In [ ]:
def is_even(n):
    return n % 2 == 0

evens = (n for n in natural_numbers() if is_even(n))
for n in evens:
    print(n, end=", ")
    if n >= 20: 
        break

Again, Python also has a dedicated `filter` function.

In [ ]:
def is_even(n):
    return n % 2 == 0

evens = filter(is_even, natural_numbers())
print(evens)
for n in evens:
    print(n, end=", ")
    if n >= 20: 
        break

## Reduce

`reduce` applies a function of two arguments ($f:R^2 \to R$) cumulatively to the items of a sequence,
from left to right, so as to reduce the sequence to a single value.

`reduce` is part of the `functools` module

In [ ]:
from functools import reduce
import operator

The easiest example is a replacement for `sum`:

In [ ]:
reduce(operator.add, range(10))

What about coding a product equivalent of `sum`?

In [ ]:
reduce(operator.mul, range(1, 10))

Another example: find intersection of several lists.

In [ ]:
lists = [[1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7]]

reduce(set.intersection, (set(x) for x in lists))

## Bonus example

Calculating the Fibonacci series up to the n-th element can also be done with `reduce`. Here we can set the initial seed to be different from the first element of the sequence, and we ignore the values from the input sequence as we always assign the "right" value to `_`:

In [ ]:
def fib_reducer(x, _):
    return x + [x[-1] + x[-2]]

def fib(n):
    return reduce(fib_reducer, range(n - 2), [0, 1])
fib(10)

## Exercise: all

Use `reduce` and something from the `operator` module to write the function `my_all(iterable)` that returns `True` if all elements of `iterable` are `True` and `False` otherwise.

**Note** Python already has an `all` function, so you can compare your results to it.

In [ ]:
seq1 = [True,  True, True]
seq2 = [True, False, True]

def myall(iterable):
    # Your code here
    pass


assert myall(seq1) == all(seq1)
assert myall(seq2) == all(seq2)

**Note** that `all` (or `any`) is implemented with a short-circuit so that once it sees a `False` (or `True`) it stops the iteration.

This can be much faster if a `False` appears early on in the sequence:

In [ ]:
data = [False] + [True] * 100000

%timeit -n 10 myall(data)
%timeit -n 10 all(data)

# Solutions

## Solution: Leap year

In [ ]:
leap_years = [year for year in range(2020) if year % 400 == 0 or (year % 4 == 0 and year % 100 !=0)]
print(len(leap_years))

## Solution: secret 

Note that `code.get` is only called when `join` starts pulling elements from `decoded`.

In [ ]:
decoded = (code.get(c, c)  for c in secret)
joined = str.join('', decoded) 
print(joined)

## Solution: factors

In [ ]:
def factors(n):
    k = 2
    while n > 1:
        if n % k == 0:
            yield k
            n //= k
        else:
            k += 1

## Solution: Fibonacci

In [ ]:
def fibonacci():
    prev = 0
    yield prev
    current = 1
    yield current
    while True:
        prev, current = current, prev + current
        yield current

## Solution: all

In [ ]:
def myall(iterable):
    booleans = (bool(e) for e in iterable)
    return reduce(operator.and_, booleans)

# References
- Some code and ideas were taken from [CS1001.py](https://github.com/yoavram/CS1001.py)
- Some code and ideas were taken from [Code like a Pythonista](http://python.net/~goodger/projects/pycon/2007/idiomatic/presentation.html)
- The [itertools](https://docs.python.org/3.5/library/itertools.html) module has some functions and tools for creating iterators.
- The [operator](https://docs.python.org/3.5/library/operator.html) module has more functions that emulate operators to used in map, reduce, etc.
- [Functional programming HOWTO](https://docs.python.org/3.5/howto/functional.html) has some more information, as the comprehensions, iterators, map-reduce and filter are the building blocks of functional programming.
- The [toolz](http://toolz.readthedocs.org/) package has many more tools to use in a functional programming style.
- Read Chapter 14 in the [Fluent Python](http://shop.oreilly.com/product/0636920032519.do) book by Luciano Ramalho for a deep dive into Iterators and everything related.

## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com).

The notebook was written using [Python](http://python.org/) 3.7.
Dependencies listed in [environment.yml](../environment.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)